In [1]:
from bs4 import BeautifulSoup 
import numpy as np
import requests
import pandas as pd
import re
import math
import statistics as st
import time
from operator import itemgetter
import datetime
import os
import collections

In [2]:
url = input()
res = requests.get(url)
res.encoding = res.apparent_encoding
soup = BeautifulSoup(res.content,features='html.parser')

https://race.netkeiba.com/race/shutuba.html?race_id=202205030411


In [3]:
horse_list = soup.find_all('tr', attrs={'class':'HorseList'})

In [5]:
horse_name = []
horse_url = []
umaban = []
wakuban = []
sex = []
age = []
kinryo = []
jockey = []
weight = []
weight_2 = []



for i in horse_list:
    try:
        wakuban.append(int(i.find('td', attrs={'class':'Waku Txt_C'}).find('span').text))
        umaban.append(int(i.find('td', attrs={'class':'Umaban Txt_C'}).text))
    except ValueError:
        wakuban.append(i.find('td', attrs={'class':'Waku Txt_C'}).find('span').text)
        umaban.append(i.find('td', attrs={'class':'Umaban Txt_C'}).text)
        
    horse_name.append(i.find('span', attrs={'class':'HorseName'}).text)
    horse_url.append(i.find('a').get('href'))
    for j, k in enumerate(i.find('td', attrs={'class':'Barei Txt_C'}).text):
        if j == 0:
            sex.append(k)
        else:
            age.append(int(k))
    kinryo.append(float(i.find_all('td', attrs={'class':'Txt_C'})[3].text))
    jockey.append(i.find('td', attrs={'class':'Jockey'}).text.strip())
    try:
        weight.append(int(i.find('td', attrs={'class':'Weight'}).text))
        weight_2.append(int(i.find('td', attrs={'class':'Weight'}).text))
    except ValueError:
        weight.append(i.find('td', attrs={'class':'Weight'}).text)
        weight_2.append(i.find('td', attrs={'class':'Weight'}).text)
    
    
    
    

レースページから馬の情報を引っ張てこれるようになった。
これからここの馬のページから馬の血統、前走の成績を引っ張てくる

In [45]:
papa, mama, mamapapa, pre_race, pre_race_result, pre_race_pop, corner_rank = [], [], [], [], [], [], []

In [46]:
for i in horse_url:
    res = requests.get(i)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.content,features='html.parser')
    #血統の取得
    blood_table = soup.find('table', attrs={'class':'blood_table'})
    papa.append(blood_table.find_all('td', attrs={'class':'b_ml'})[0].text.strip())
    mama.append(blood_table.find_all('td', attrs={'class':'b_fml'})[1].text.strip())
    mamapapa.append(blood_table.find_all('td', attrs={'class':'b_ml'})[2].text.strip())
    #前走情報の取得
    race_results = soup.find('table', attrs={'class':'db_h_race_results nk_tb_common'}).find('tbody')
    if race_results.find_all('td')[11] != '除':
        pre_race.append(race_results.find_all('td')[4].text)
        pre_race_result.append(race_results.find_all('td')[11].text)
        pre_race_pop.append(race_results.find_all('td')[10].text)
        corner_rank.append(race_results.find_all('td')[20].text)
    else:
        pre_race.append(race_results.find_all('td')[32].text)
        pre_race_result.append(race_results.find_all('td')[39].text)
        pre_race_pop.append(race_results.find_all('td')[38].text)
        corner_rank.append(race_results.find_all('td')[48].text)

In [49]:
pre_race

['都大路S(L)',
 '東京優駿(G1)',
 'ダービー卿チャレンジ(G3)',
 'ヴィクトリアマイル(G1)',
 '湘南S(3勝クラス)',
 '京王杯スプリングC(G2)',
 'ダービー卿チャレンジ(G3)',
 '新潟大賞典(G3)',
 'ダービー卿チャレンジ(G3)',
 'アメジストS(3勝クラス)',
 '安土城S(L)',
 '飛鳥S(3勝クラス)']

0 2022/05/14
1 3中京3
2 晴
3 11
4 都大路S(L)
5 


6 7
7 6
8 6
9 4.6
10 4
11 6
12 
鮫島克駿

13 57
14 芝2000
15 稍
16 

**


17 2:00.8
18 0.3
19 

**


20 3-3-3-3
21 37.6-34.6
22 34.6
23 512(-10)
24 


25 


 

26 シフルマン
27  
28 2022/02/27
29 2中山2
30 晴
31 11
32 中山記念(G2)
33 


34 16
35 2
36 3
37 26.9
38 8
39 4
40 
田辺裕信

41 56
42 芝1800
43 良
44 

**


45 1:47.2
46 0.8
47 

**


48 12-12-13-11
49 35.2-37.3
50 36.1
51 522(+12)
52 


53 


 

54 パンサラッサ
55 1,000.0
56 2021/06/13
57 3東京4
58 曇
59 11
60 エプソムC(G3)
61 


62 18
63 7
64 14
65 34.7
66 12
67 12
68 
野中悠太

69 56
70 芝1800
71 良
72 

**


73 1:46.5
74 1.4
75 

**


76 6-5-8
77 35.3-34.9
78 35.8
79 510(+10)
80 


81 


 

82 ザダル
83  
84 2021/05/15
85 3中京3
86 晴
87 11
88 都大路S(L)
89 


90 11
91 1
92 1
93  
94  
95 除
96 
川田将雅

97 57
98 芝2000
99 良
100 

**


101  
102  
103 

**


104  
105 36.6-34.5
106  
107 504(+4)
108 


109 


 

110 マウントゴールド
111  
112 2020/12/20
113 5中山6
114 晴
115 11
116 ディセンバーS(L)
117 


118 15
119 6
120 10
121 2.3
122 1
123 3
124 
田辺裕信